In [9]:
import numpy as np
import random
import matplotlib as plt

In [43]:
#parameters of distribution
Ec = 5
Dc = 0.1
scale = Dc**0.5

#parameters of utility
sigma = 1
S = 1

#parameters of firms
f = 0.01 # fixed costs
ef = 0.025 # entrance costs
r = 0.1 #return of interest

In [52]:
class ApsundNokeModel():
    def __init__(self, distr, params, sigma, S, f, ef, r, alpha):
        self.distr = distr
        self.params = params
        self.Ec = Ec
        self.Dc = Dc
        self.scale = Dc**0.5
        self.sigma = sigma
        self.S = S
        self.f = f
        self.ef = ef
        self.r = r
        self.alpha = alpha

        self.C = []
    
    def Sigma2(self):
        C = self.C
        sigma = self.sigma
        return (2+sigma*np.sum(C))/(2+sigma*len(C))
    
    def expectedProfit_montecarlo(self, n = 20000):
        loc = self.Ec
        scale = self.scale
        f = self.f

        sigma2 = self.Sigma2()
        answ = 0
        for i in range(n):
            cc = np.random.normal(loc,scale)
            answ -= f
            if cc>= sigma2:
                answ += 1/8*(sigma2-cc)**2
        return answ/n

    def NPV_for_entrant(self, n = 20000):
        r = self.r
        return ((1+r)/r) * self.expectedProfit_montecarlo(n)
    
    def scenario(self, epochs = 1000, out = False, file = None):
        alpha = self.alpha
        f = self.f
        r = self.r
        
        #file for logs
        if file:
            log = open(file, "w")
        

        #zero epoch
        self.C = np.abs(self.distr(*self.params, size = 10))
        
        for epoch in range(epochs):
            NPV1 = self.NPV_for_entrant()
            EP = self.expectedProfit_montecarlo()

            #entrance stage
            n_ent = 0
            while NPV1 > ef:
                c = np.abs(self.distr(*self.params, size = 1))
                self.C = np.append(self.C, c)
                n_ent += 1
                NPV1 = self.NPV_for_entrant()

            #recounting stage
            for i in range(len(self.C)):
                a = random.random()
                if a > alpha:
                    self.C[i] = np.abs(self.distr(*self.params))
            
            # sort of C
            self.C = np.flip(np.sort(self.C))

            # exit stage
            EP = self.expectedProfit_montecarlo()
            sigma2 = self.Sigma2()
            n_ex = 0
            i = 0
            P_fact = []
            while (i < len(self.C)):
                P = - f
                if sigma2 > self.C[i]:
                    P = 1/8*(sigma2 - self.C[i])**2 - f
                P_fact.append(P)
                NPV2 = (1+r)/(1+r-alpha) * P + (1/r - alpha/(1+r-alpha))*EP
                if NPV2 < 0:
                    self.C = np.delete(self.C, i)
                    n_ex += 1
                    EP = self.expectedProfit_montecarlo()
                    sigma2 = self.Sigma2()
                else:
                    i += 1
            m = len(self.C)
            Mean_P = np.mean(P_fact)

            if out:
                print(f"epoch{epoch}, number of firms: {m}, entrats: {n_ent}, exiters: {n_ex}, EP: {EP}, NPV1: {NPV1}, EP_fact: {Mean_P}")
            if file:
                log.write(f"epoch{epoch}, number of firms: {m}, entrats: {n_ent}, exiters: {n_ex}, EP: {EP}, NPV1: {NPV1}, EP_fact: {Mean_P} \n")

In [53]:
Noke = ApsundNokeModel(distr=np.random.normal,
                       params=(Ec, scale),
                       sigma=sigma,
                       S=S,
                       f=f,
                       ef=ef,
                       r=r,
                       alpha=0.5
                       )

In [54]:
Noke.scenario(out=True, file="logs.txt")

epoch0, number of firms: 44, entrats: 34, exiters: 0, EP: 0.004323107094512911, NPV1: 0.023932670757574173, EP_fact: -0.0071043863575145
epoch1, number of firms: 51, entrats: 11, exiters: 4, EP: 0.002241787336287622, NPV1: 0.021855885194607315, EP_fact: -0.008736636423037418
epoch2, number of firms: 53, entrats: 3, exiters: 1, EP: 0.002337667049311267, NPV1: 0.021703117096589, EP_fact: -0.008880829686534987
epoch3, number of firms: 52, entrats: 0, exiters: 1, EP: 0.0020482103541073025, NPV1: 0.023237670743189866, EP_fact: -0.00807307013239381
epoch4, number of firms: 50, entrats: 0, exiters: 2, EP: 0.002752325144981375, NPV1: 0.020101387445923127, EP_fact: -0.008509448594820345
epoch5, number of firms: 51, entrats: 1, exiters: 0, EP: 0.002965999068877382, NPV1: 0.02479715413891915, EP_fact: -0.007974103981831999
epoch6, number of firms: 57, entrats: 7, exiters: 1, EP: 0.0024617927921354132, NPV1: 0.01586496181178461, EP_fact: -0.008434916880059025
epoch7, number of firms: 58, entrats: 

KeyboardInterrupt: 

In [118]:
def Sigma2(C, sigma):
    return (2+sigma*np.sum(C))/(2+sigma*len(C))


def Ec_montecarlo(loc, scale, sigma2, n=20000):
    answ = 0
    for i in range(n):
        cc = np.random.normal(loc, scale)
        if cc >= sigma2:
            answ += cc
    return answ/n

def Ec2_montecarlo(loc, scale, sigma2, n = 20000):
    answ = 0
    for i in range(n):
        cc = np.random.normal(loc, scale)
        if cc >= sigma2:
            answ += cc**2
    return answ/n

def expectedProfit(C, sigma, f, loc, scale):
    sigma2 = Sigma2(C, sigma)
    return 1/8*(sigma2**2 -2*sigma2 * Ec_montecarlo(loc, scale, sigma2) + Ec2_montecarlo(loc, scale, sigma2)) - f

def expectedProfit_montecarlo(C, sigma, f, loc, scale):
    sigma2 = Sigma2(C, sigma)
    answ = 0
    for i in range(20000):
        cc = np.random.normal(loc,scale)
        answ -= f
        if cc>= sigma2:
            answ += 1/8*(sigma2-cc)**2
    return answ/20000

def NPV_for_entrant(C, r, sigma, f, loc, scale):
    sigma2 = Sigma2(C, sigma)
    return ((1+r)/r) * expectedProfit_montecarlo(C, sigma, f, loc, scale)


In [159]:
#zero epoch
C = np.abs(np.random.normal(loc = Ec, scale = Dc**0.5, size = 10))

#in each epoch only one firm can entry
alpha = 0.5

for epoch in range(1000):
    NPV1 = NPV_for_entrant(C, r, sigma, f, Ec, scale)
    EP = expectedProfit_montecarlo(C, sigma, f, Ec, scale)
    sigma2 = Sigma2(C, sigma)
    n_ent = 0
    while NPV1 > ef:
        c = np.random.normal(Ec, scale, 1)
        C = np.append(C, c)
        n_ent += 1
        NPV1 = NPV_for_entrant(C, r, sigma, f, Ec, scale)
    #print(EP)

    for i in range(len(C)):
        a = random.random()
        if a > alpha:
            C[i] = np.random.normal(Ec, scale)
    
    C = np.flip(np.sort(C))

    EP = expectedProfit_montecarlo(C, sigma, f, Ec, scale)
    sigma2 = Sigma2(C, sigma)
    n_ex = 0
    i = 0
    P_fact = []
    while (i < len(C)):
        P = -f
        if sigma2 > C[i]:
            P = 1/8*(sigma2 - C[i])**2 - f
        P_fact.append(P)
        #print(P)
        NPV2 = (1+r)/(1+r-alpha) * P + (1/r - alpha/(1+r-alpha))*EP
        #print(P, NPV2)
        if NPV2 < 0:
            C = np.delete(C, i)
            n_ex += 1
            EP = expectedProfit_montecarlo(C, sigma, f, Ec, scale)
        else:
            i += 1
    m = len(C)
    EP_fact = np.mean(P_fact)
    print(f"epoch{epoch}, number of firms: {m}, entrats: {n_ent}, exiters: {n_ex}, EP: {EP}, NPV1: {NPV1}, sigma2: {sigma2}, EP_fact: {EP_fact}")

print(len(C))

epoch0, number of firms: 487, entrats: 477, exiters: 0, EP: 0.0021825189204853586, NPV1: 0.02404928424820122, sigma2: 4.979260084821722, EP_fact: 0.0007427202587150237
epoch1, number of firms: 488, entrats: 1, exiters: 0, EP: 0.002749040508431522, NPV1: 0.02268102325771654, sigma2: 4.969908443675145, EP_fact: 0.0012630220614927251
epoch2, number of firms: 495, entrats: 7, exiters: 0, EP: 0.00208017065396085, NPV1: 0.02157873864083134, sigma2: 4.977274210007528, EP_fact: 2.9522425462040578e-05
epoch3, number of firms: 495, entrats: 0, exiters: 0, EP: 0.003074504368320854, NPV1: 0.023603824719035448, sigma2: 4.960398042060248, EP_fact: 0.001298308912187507
epoch4, number of firms: 758, entrats: 264, exiters: 1, EP: 0.0023434735099535347, NPV1: 0.022637955934282045, sigma2: 4.984068774537921, EP_fact: 0.0015749591623869339
epoch5, number of firms: 758, entrats: 0, exiters: 0, EP: 0.0032241656189464997, NPV1: 0.019536224748249224, sigma2: 4.967826474732854, EP_fact: 0.0016320811582460372
e